In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/chicago_taxis_ii.csv') #In short, we're doing this. See Readme.
print(df.shape)
df.head()

(38349, 13)


,taxi_id,trip_start_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company
0,008dda45db57cb6daa679a86ce8c8149ddc05446d545b1...,2015-01-14 14:30:00.000000 UTC,321,0.9,32.0,32.0,5.65,1.00,NaN,0.0,6.65,Credit Card,Yellow Cab
1,008dda45db57cb6daa679a86ce8c8149ddc05446d545b1...,2015-01-16 03:30:00.000000 UTC,688,4.2,24.0,22.0,12.25,2.45,NaN,0.0,14.70,Credit Card,Yellow Cab
2,008dda45db57cb6daa679a86ce8c8149ddc05446d545b1...,2015-01-01 01:30:00.000000 UTC,781,3.9,4.0,6.0,11.65,2.91,NaN,0.0,14.56,Credit Card,Yellow Cab
3,008dda45db57cb6daa679a86ce8c8149ddc05446d545b1...,2015-01-24 06:00:00.000000 UTC,104,0.4,6.0,6.0,3.85,15.71,NaN,0.0,19.56,Credit Card,Yellow Cab
4,008dda45db57cb6daa679a86ce8c8149ddc05446d545b1...,2015-01-06 22:00:00.000000 UTC,176,0.8,6.0,6.0,4.85,1.00,NaN,0.0,5.85,Credit Card,Yellow Cab


In [3]:
df['company'].value_counts() #Wow, despite trying to grab 40k guys (38.3k per the 10 mb download limit) we still just hav
#Yellow cab. Hmm... Yeah, we'll drop this column. I still want to examine cross-company business, but likely the human element,
#the part that's truly worthy of a 'tip', if at all, is best exemplifid here. So, let us finally be satisifed with this dataset
#and continue.

company
Yellow Cab    38349
Name: count, dtype: int64

In [4]:
print(df.shape[0]/(len(df['taxi_id'].unique())*31)) #Hmm, so the average taxi driver is giving six rides a day. Keep this in mind for later.

5.947425558312655


In [5]:
#On principal I'll redo all of the coding checks I usually do, yet I'll save commentary for something new.
df.columns

Index(['taxi_id', 'trip_start_timestamp', 'trip_seconds', 'trip_miles',
       'pickup_community_area', 'dropoff_community_area', 'fare', 'tips',
       'tolls', 'extras', 'trip_total', 'payment_type', 'company'],
      dtype='object')

In [6]:
print(df.isnull().sum()) #Oh wow... all of the tolls are nulls... looks like we'll just be dropping that column then.

#Yeah, let's go ahead and borrow inspiration from the below code and go ahead and knock out a totally nulled row...
    #Perhaps a fail safe if that's the target variable? Nah, I don't care what the board says - if the target is null...
    #they're dull and drop them.

for i in df.columns:
    if df[i].isnull().sum() == df.shape[0]:
        df = df.drop([i], axis=1)
        print(f"Hence we just dropped {i} as sadly, without exception, that entire column was nulled...")

taxi_id                       0
trip_start_timestamp          0
trip_seconds                  0
trip_miles                    0
pickup_community_area       342
dropoff_community_area     1261
fare                          0
tips                          0
tolls                     38349
extras                        0
trip_total                    0
payment_type                  0
company                       0
dtype: int64
Hence we just dropped tolls as sadly, without exception, that entire column was nulled...


In [7]:
#Added a new feature to also take care of 1 dimensional fields - no need to keep those.

for i in df.columns:
    if len(df[i].unique()) == df.shape[0] or len(df[i].unique()) == 1:
        df = df.drop([i], axis=1) #Not adding anything to our model... Hmm, now that I think about it, let's make this better:
        print(f"Just dropped {i} as it wasn't adding any value whatsoever to our model.")
    else:
        print(f"Well, {i} has {len(df[i].unique())} unique fields, so we'll keep it...for now!")

Well, taxi_id has 208 unique fields, so we'll keep it...for now!
Well, trip_start_timestamp has 2907 unique fields, so we'll keep it...for now!
Well, trip_seconds has 3137 unique fields, so we'll keep it...for now!
Well, trip_miles has 325 unique fields, so we'll keep it...for now!
Well, pickup_community_area has 68 unique fields, so we'll keep it...for now!
Well, dropoff_community_area has 73 unique fields, so we'll keep it...for now!
Well, fare has 313 unique fields, so we'll keep it...for now!
Well, tips has 906 unique fields, so we'll keep it...for now!
Well, extras has 81 unique fields, so we'll keep it...for now!
Well, trip_total has 1665 unique fields, so we'll keep it...for now!
Well, payment_type has 2 unique fields, so we'll keep it...for now!
Just dropped company as it wasn't adding any value whatsoever to our model.


In [8]:
df['taxi_id'] = df['taxi_id'].replace(list(df['taxi_id'].unique()),[f"taxi_{i}" for i in range(df['taxi_id'].unique().shape[0])])

df['start_date'] = pd.to_datetime(df['trip_start_timestamp']).dt.date
df['start_time'] = pd.to_datetime(df['trip_start_timestamp']).dt.time

df = df.drop(['trip_start_timestamp'], axis = 1)

In [11]:
print(df['trip_seconds'].isnull().sum()/df.shape[0])
#print(df[df['trip_seconds'].isnull()]['trip_miles'].describe())
#df['trip_seconds'].replace(np.nan,0.0, inplace=True)
print(df[df['trip_seconds']==0].shape[0]/df.shape[0])
df['trip_seconds'].describe()

0.0
0.0


count    38349.000000
mean       817.502047
std        642.643298
min          1.000000
25%        388.000000
50%        614.000000
75%       1043.000000
max       8319.000000
Name: trip_seconds, dtype: float64

Now there we go! Still skewed to the right, but a lot more of what I'd expect with actual taxi rides! Ie an over 10 minute median and 13 minute mean taxi ride... sounds a lot more realistic. 

In [12]:
print(df['trip_miles'].isnull().sum()/df.shape[0])
#print(df[df['trip_miles'].isnull()]['trip_seconds'].describe())
#df['trip_miles'].replace(np.nan,0.0, inplace=True)
print(df[df['trip_miles']==0].shape[0]/df.shape[0])
df['trip_miles'].describe()

0.0
0.0


count    38349.000000
mean         4.729565
std          5.610072
min          0.100000
25%          1.100000
50%          2.200000
75%          5.500000
max         63.400000
Name: trip_miles, dtype: float64

Similarily, trip miles seem reasonble. Of interest would be to consider factors such as location re traffic and the like as notice that here the mean is more than half of the mean!

In [16]:
print(df[df['pickup_community_area'].notnull()].shape[0])
print(df[df['dropoff_community_area'].notnull()].shape[0])
print(len(df['pickup_community_area'].unique()), len(df['dropoff_community_area'].unique())) #Counting nulls

38007
37088
68 73


In [17]:
df['fare'].describe()

count    38349.000000
mean        12.987318
std         10.375461
min          3.250000
25%          6.050000
50%          8.450000
75%         14.850000
max        117.650000
Name: fare, dtype: float64

Continuing the theme of right skews, yet reasonable. Quite happy with this.

In [9]:
df.head()

,taxi_id,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,extras,trip_total,payment_type,start_date,start_time
0,taxi_0,321,0.9,32.0,32.0,5.65,1.00,0.0,6.65,Credit Card,2015-01-14,14:30:00
1,taxi_0,688,4.2,24.0,22.0,12.25,2.45,0.0,14.70,Credit Card,2015-01-16,03:30:00
2,taxi_0,781,3.9,4.0,6.0,11.65,2.91,0.0,14.56,Credit Card,2015-01-01,01:30:00
3,taxi_0,104,0.4,6.0,6.0,3.85,15.71,0.0,19.56,Credit Card,2015-01-24,06:00:00
4,taxi_0,176,0.8,6.0,6.0,4.85,1.00,0.0,5.85,Credit Card,2015-01-06,22:00:00
